In [2]:
import numpy as np
import pylab as plt
import h5py
%matplotlib qt

In [18]:
from json import loads
#settings_file = 'D:\\vnd_spbu\\pilot\\mu5days\\vnd_spbu_5days.json'
#settings_file = 'D:\\vnd_spbu\\mock\\vnd_spbu_5days.json'
settings_file = 'C:\\Users\\nsmetanin\\Desktop\\results\\vnd_spbu_5days.json'
with open(settings_file, 'r', encoding="utf8") as f:
    settings = loads(f.read())

In [19]:
experiment = settings['subjects'][0][0]
experiment

'a6_d1_4states_03-27_20-43-52'

In [59]:
from collections import OrderedDict

from scipy.signal import welch

from pynfb.postprocessing.utils import load_rejections, get_info, add_data, fft_filter, dc_blocker, get_colors

with h5py.File('{}\\{}\\{}'.format(settings['dir'], experiment, 'experiment_data.h5')) as f:
    fs, channels, p_names = get_info(f, settings['drop_channels'])
    xx = np.concatenate([fft_filter(f['protocol{}/raw_data'.format(j + 1)][:], fs, (2, 48)) for j in [0,1,3]])
    
    plt.plot(*welch(f['protocol{}/raw_data'.format(1)][:, channels.index('C3')], fs, nperseg=1000))
    plt.plot(*welch(f['protocol{}/raw_data'.format(2)][:, channels.index('C3')], fs, nperseg=1000))
    plt.plot(*welch(f['protocol{}/raw_data'.format(3)][:, channels.index('C3')], fs, nperseg=1000))
    plt.plot(*welch(f['protocol{}/raw_data'.format(4)][:, channels.index('C3')], fs, nperseg=1000))
    plt.legend(['Close', 'Open', 'Left', 'Right'])
    plt.show()

    rejections, top_alpha, top_ica = load_rejections(f, reject_alpha=False)
    
    raw = OrderedDict()
    for j, name in enumerate(p_names):
        x = f['protocol{}/raw_data'.format(j + 1)][:]
        #x = np.dot(x, rejections)
        x = dc_blocker(x)
        x = fft_filter(x, fs, (2, 48))
        raw = add_data(raw, name, x, j)

fs: 500
all labels 32: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'Ft9', 'Fc5', 'Fc1', 'Fc2', 'Fc6', 'Ft10', 'T7', 'C3', 'Cz', 'C4', 'T8', 'Tp9', 'Cp5', 'Cp1', 'Cp2', 'Cp6', 'Tp10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2']
selected channels 32: ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'Ft9', 'Fc5', 'Fc1', 'Fc2', 'Fc6', 'Ft10', 'T7', 'C3', 'Cz', 'C4', 'T8', 'Tp9', 'Cp5', 'Cp1', 'Cp2', 'Cp6', 'Tp10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2']
protocol_names: ['Close', 'Open', 'Left', 'Right']


KeyError: 'Unable to open object (Component not found)'

In [38]:
cm = get_colors()
t = 0
ch_ind = channels.index('C3')
for j_p, (name, x) in enumerate(raw.items()):
    print(name)
    time = np.arange(t, t + len(x)) / fs
    color = cm[''.join([i for i in name.split()[1] if not i.isdigit()])]
    xf = fft_filter(x[:, ch_ind], fs, band=(8, 14))
    pow_ = 2
    plt.plot(time, xf**pow_, c=color, alpha=0.8)
    plt.plot(time, x[:, ch_ind]**pow_, linewidth=1, c=color, alpha=0.5)
    plt.plot(time, time*0 + xf.std()**pow_, c=color, linewidth=5)
    plt.plot(time, time*0 + np.percentile(xf**pow_, 90), '--', c=color, linewidth=5)
    t += len(x)

1. Closed
1. Opened
2. Right
2. Left
3. Baseline


4. FB1
5. Rest
6. FB2
7. Rest
8. FB3
9. Rest
10. FB4


11. Rest
12. FB5
13. Rest
14. Right
14. Left


15. Baseline


In [34]:
from pynfb.protocols.ssd.topomap_selector_ica import ICADialog


#xx = np.concatenate([raw['1. Closed'], raw['1. Opened'], raw['2. Right'], raw['2. Left'],  raw['14. Right'], raw['14. Left'], raw['3. Baseline']])

#xx = np.concatenate([raw['1. Opened'][len(raw['1. Opened'])//2:], raw['2. Right']])
#xx = np.concatenate([raw['1. Opened'], raw['2. Right'], raw['2. Left']])
#xx = np.concatenate([raw['1. Closed'][len(raw['1. Opened'])//2:], raw['1. Opened'][len(raw['1. Opened'])//2:], raw['2. Right']])

rejection, spatial, unmixing = ICADialog.get_rejection(xx, channels, fs, mode='csp', states=0.7)

CSP 3 using
ICA/CSP time elapsed = 0.08800888061523438s


Table drawing time elapsed = 11.6721670627594s


ValueError: too many values to unpack (expected 3)

In [32]:
fs

500